In [1]:
import os

train_src_dir = "./src"
os.makedirs(train_src_dir, exist_ok=True)

In [4]:
!pip install azure-ai
!pip install azure-ai-ml

ERROR: Could not find a version that satisfies the requirement azure-ai (from versions: none)
ERROR: No matching distribution found for azure-ai


In [ ]:
!wget https://azuremlexamples.blob.core.windows.net/datasets/credit_card/default%20of%20credit%20card%20clients.csv

In the next cell, enter your Subscription ID, Resource Group name and Workspace name. To find these values:

1.    In the upper right Azure Machine Learning studio toolbar, select your workspace name.
2.    Copy the value for workspace, resource group and subscription ID into the code.
3.    You'll need to copy one value, close the area and paste, then come back for the next one.

In [65]:
# Handle to the workspace
from azure.ai.ml import MLClient

# Authentication package
# from azure.identity import DefaultAzureCredential

# credential = DefaultAzureCredential()

# ml_client = MLClient(
#     credential=credential,
#     subscription_id="827dbc18-97d7-4a9d-91d1-a0d0d15c9603",
#     resource_group_name="blazejtest",
#     workspace_name="blazejamlworkshop",
# )

from azureml.core import Workspace
ws = Workspace.from_config()


{
  "source": [
    "('workspaceblobstore', 'UI/2022-11-04_145559_UTC/')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ParseDelimited",
    "DropColumns",
    "SetColumnTypes"
  ],
  "registration": {
    "id": "ef696e23-0ad7-41a6-a4c0-e9caec169702",
    "name": "credit_cards",
    "version": 1,
    "workspace": "Workspace.create(name='blazejamlworkshop', subscription_id='827dbc18-97d7-4a9d-91d1-a0d0d15c9603', resource_group='blazejtest')"
  }
}

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,29996,220000,1,3,1,39,0,0,0,0,...,88004,31237,15980,8500,20000,5003,3047,5000,1000,0
29996,29997,150000,1,3,2,43,-1,-1,-1,-1,...,8979,5190,0,1837,3526,8998,129,0,0,0
29997,29998,30000,1,2,2,37,4,3,2,-1,...,20878,20582,19357,0,0,22000,4200,2000,3100,1
29998,29999,80000,1,3,1,41,1,-1,0,0,...,52774,11855,48944,85900,3409,1178,1926,52964,1804,1


azureml://subscriptions/827dbc18-97d7-4a9d-91d1-a0d0d15c9603/resourcegroups/blazejtest/workspaces/blazejamlworkshop/datastores/workspaceblobstore/paths/UI/2022-11-04_141501_UTC/credit_cards(2).csv: Unsupported scheme ‘azureml’.


## Good, old fashioned model training

In [8]:
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

file_name = 'default of credit card clients.csv'
TEST_TRAIN_RATIO = 0.2


credit_df = pd.read_csv(file_name, header=1, index_col=0)

train_df, test_df = train_test_split(
    credit_df,
    test_size=TEST_TRAIN_RATIO,
)
###################
#</prepare the data>
###################

##################
#<train the model>
##################
# extracting the label column
y_train = train_df.pop("default payment next month")

# convert the dataframe values to array
X_train = train_df.values

# extracting the label column
y_test = test_df.pop("default payment next month")

# convert the dataframe values to array
X_test = test_df.values

print(f"Training with data of shape {X_train.shape}")

clf = GradientBoostingClassifier(
)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

Training with data of shape (24000, 23)


In [12]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.78      1.00      0.88      4680
           1       0.00      0.00      0.00      1320

    accuracy                           0.78      6000
   macro avg       0.39      0.50      0.44      6000
weighted avg       0.61      0.78      0.68      6000



## But what if dataset changes frequently?

* What if we want to make sure that model performs well on new test set?
* What if we want to use new data to retrain our model?
* What if we want to compare how our model performs over time, when new data is added?

**Of course we could code / script everything, but why not make our lives easier - let's abstract the dataset!**

https://raw.githubusercontent.com/unit8co/azureml-agh-workshop/main/credit_cards.csv

## 

In [ ]:
from azureml.core import Dataset, Workspace

ws = Workspace.from_config()

dataset = Dataset.get_by_name(ws, name="credit_cards", version="latest")
dataset.to_pandas_dataframe()